In [1]:
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString,mapping
import WGS1984

from tqdm import tqdm
import matplotlib.pyplot as plt
import haversine as hs

In [2]:
def point_to_line(df):
    df = df.reset_index()
    df = df.sort_values("time")
    df.loc[0,"time_seg"] = 0
    df.loc[0,"distance"] = 0 
    
    print(df["车牌"].unique())
    
    for i in range(len(df)-1):
        df.loc[i+1,"time_seg"] = (df.loc[i+1,"time"] - df.loc[i,"time"]).total_seconds()
        df.loc[i+1,"distance"] = hs.haversine((df.loc[i+1,"y"],df.loc[i+1,"x"]),(df.loc[i,"y"],df.loc[i,"x"]))*1000

        if float(df.loc[i,"time_seg"])*50 < float(df.loc[i,"distance"]):
            df.loc[i,"sb"] = 1
        else:
            df.loc[i,"sb"] = 0
        
    try:
        df = df[df['sb']==0]
    except:
        df = df

    return LineString(df.sort_values("time")[["x","y"]].values)

In [3]:
df = pd.read_csv("/Users/creative/NutstoreCloudBridge/我的坚果云/文件转存区/Win/data/20161115.csv")

In [3]:
df = pd.read_excel("/Users/creative/Desktop/问题数据位置点.xlsx")

In [4]:
#找到为1的车牌
count = {'车牌':df["车牌"].value_counts().index,'numbers':df["车牌"].value_counts().values}
df_count = pd.DataFrame(count)
#连接属性表
adf = pd.merge(df,df_count,on="车牌")

In [5]:
#删除为1的车牌
adf = adf[adf['numbers'] > 2]

In [6]:
#转换时间序列
adf['time']=pd.to_datetime(adf["time"]) 

In [7]:
#点集转线
line = (adf
           .groupby("车牌")
           .apply(point_to_line)
           .to_frame(name="geometry")
           .pipe(gpd.GeoDataFrame,crs="EPSG:4326"))

['粤BAB218']
['粤BAD542']
['粤BAE971']
['粤BAF029']
['粤BAF669']
['粤BAJ751']
['粤BAL262']
['粤BAL626']
['粤BAL641']
['粤BAL646']
['粤BAL676']
['粤BAN059']
['粤BAN157']
['粤BAN358']
['粤BAN783']
['粤BAP003']
['粤BAQ537']
['粤BAU522']
['粤BB9937']
['粤BBH619']
['粤BBP970']
['粤BBQ017']
['粤BBQ139']
['粤BBT093']
['粤BC0918']
['粤BCL317']
['粤BCL486']
['粤BCR068']
['粤BCR469']
['粤BCR908']
['粤BCU923']
['粤BCW819']
['粤BCW861']
['粤BCW863']
['粤BCW867']
['粤BCW941']
['粤BCW951']
['粤BCX340']
['粤BCY116']
['粤BCY123']
['粤BCY233']
['粤BCY350']
['粤BCY357']
['粤BCY429']
['粤BCY668']
['粤BCY685']
['粤BCY729']
['粤BCY962']
['粤BCZ247']
['粤BCZ310']
['粤BDB205']
['粤BDB685']
['粤BDC146']
['粤BDC157']
['粤BDC473']
['粤BDC479']
['粤BDC486']
['粤BDC495']
['粤BDC820']
['粤BDE672']
['粤BDE675']
['粤BDE678']
['粤BDE685']
['粤BDG103']
['粤BDH369']
['粤BDK223']
['粤BDY550']
['粤BE3811']
['粤BE8242']
['粤BE9020']
['粤BEM681']
['粤BES242']
['粤BF4012']
['粤BH0212']
['粤BH6421']
['粤BJ6848']
['粤BJ7843']
['粤BJ8828']
['粤BK1757']
['粤BK2540']
['粤BK9007']
['粤BL6251']
['粤BN6943']
['粤B

In [8]:
lines = line.merge(df, on='车牌')
lines = lines.drop_duplicates('车牌')
lines = lines.drop(columns=['time', 'parking_time','status','x','y'])
lines = lines.reset_index()

In [10]:
lines.to_file('/Users/creative/Desktop/问题轨迹.shp',encoding = 'gb18030') 